In [1]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

import os
from matplotlib import pyplot as plt
from gpt import GPT

from matplotlib import pyplot as plt
import numpy as np

# Autotranslation of Tao FODO Lattice to GPT
---
This example uses pytao to load a Tao FODO lattice example and then converts it to a GPT lattice.  Note this will require installing the pytao package:

see: [https://github.com/bmad-sim/pytao](https://github.com/bmad-sim/pytao)

In [2]:
from pytao import Tao

ModuleNotFoundError: No module named 'pytao'

In [ ]:
#tao = Tao('-init templates/tao/fodo/fodo.lat.bmad -noplot -external_plotting')
tao = Tao('-init templates/tao/fodo/tao.init -noplot -external_plotting')
G = GPT.from_tao(tao)
G.lattice.plot_floor(screen_alpha=0.025);

In [ ]:
from pmd_beamphysics import single_particle, ParticleGroup

from scipy.constants import physical_constants, c

MC2 = physical_constants['electron mass energy equivalent in MeV'][0]*1e6

In [ ]:
E = 150e6
p = np.sqrt(E**2 - MC2**2)

In [ ]:
Px, Py = single_particle(x=1e-3, pz=p), single_particle(y=1e-3, pz=p)
PXFILE, PYFILE = 'x_probe_particle.h5', 'y_probe_particle.h5'

Px.write(PXFILE)
Py.write(PYFILE)

In [ ]:
G.initial_particles = Px + Py

In [ ]:
G.set_variables({'tmax': 5e-8})

In [ ]:
G.run()

In [ ]:
print(G)

In [ ]:
tao.cmd('set beam_init position_file = x_probe_particle.h5');
tao.cmd(f'set beam_init n_particle = {len(Px)}');
tao.cmd('set global track_type = beam');

ix = tao.lat_list('marker::*', 'ele.ix_ele')
PGxs = [ ParticleGroup(data=tao.bunch_data(i)) for i in ix]

tao.cmd('set beam_init position_file = y_probe_particle.h5');
tao.cmd(f'set beam_init n_particle = {len(Px)}');
tao.cmd('set global track_type = beam');

ix = tao.lat_list('marker::*', 'ele.ix_ele')
PGys = [ ParticleGroup(data=tao.bunch_data(i)) for i in ix]

In [ ]:
def get_mean_value(key, PGs):
    values = np.array([PG[f'mean_{key}'] for PG in PGs])
    return values

def get_sigma_value(key, PGs):
    values = np.array([PG[f'sigma_{key}'] for PG in PGs])
    return values

fig, axs = plt.subplots(2, 1)# constrained_layout=True)
tx = G.trajectory(1, data_type='screen')
ty = G.trajectory(2, data_type='screen')

axs[0].plot(c*tx['t'], tx['x']*1e3, color='tab:blue')
axs[0].plot(c*get_mean_value('t', PGxs), 1e3*get_mean_value('x', PGxs), '--', color='tab:blue')

axs[0].plot(c*ty['t'], ty['y']*1e3, color='tab:orange')
axs[0].plot(c*get_mean_value('t', PGys), 1e3*get_mean_value('y', PGys), '--', color='tab:orange')

axs[0].set_xlabel('s (m)')
axs[0].set_ylabel('$<x, y>$ (mm)')

G.lattice.plot_floor(screen_alpha=0.025, ax=axs[-1]);

axs[0].legend(['GPT x-offset particle', 'Tao x-offset particle', 'GPT y-offset particle', 'Tao y-offset particle']);

In [ ]:
os.remove('x_probe_particle.h5')
os.remove('y_probe_particle.h5')